In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import pandas as pd
import time
import pickle

In [3]:
with open('fractionalized_nft_mevs_list', 'rb') as f: fract_nft_mevs_list = pickle.load(f)
len(fract_nft_mevs_list)

2130

In [4]:
def collect_gas_info(mevs_transactions_list: list, interval_between_iterations: float = 5.0, name_of_file: str='df') -> pd.DataFrame:
    url = 'https://etherscan.io/tx/'
    txds = mevs_transactions_list
    driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
    df = pd.DataFrame(columns=['txd', 'block', 'block_position',
                               #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth',
                               'gas_usage_by_tx', 'ether_price', 'tx_fee'])
    iteration = 0
    total_iterations = len(mevs_transactions_list)

    for txd in txds:
        driver.get(url + txd)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        possible_block = soup.findAll("div", attrs={'class': 'col-md-9'})
        possible_transaction_fee = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanGasPrice'})
        possible_ether_price = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'})
        possible_gas_tx_usage = soup.findAll('span', attrs={'data-original-title': 'The amount of GAS used by this specific transaction alone'})
        possible_blocks = soup.findAll('span', attrs={'class': 'text-dark'})

        block = possible_block[2].findAll('a')[0].text if len(possible_block) != 0 else 0
        transaction_fee = possible_transaction_fee[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '') \
            if len(possible_transaction_fee) != 0 else 0
        ether_price = possible_ether_price[0].text.replace('\n$', '').replace(' / ETH', '') \
            if len(possible_ether_price) != 0 else 0
        gas_usage_by_tx = possible_gas_tx_usage[0].text.split('(')[0].replace('\n', '').replace(' ', '') \
            if len(possible_gas_tx_usage) != 0 else 0
        blocks = possible_blocks
        block_position = blocks[len(blocks) - 1].text

        new_row = {'txd': txd,
                   'block': block,
                   'block_position': block_position,
                   'gas_usage_by_tx': gas_usage_by_tx,
                   'ether_price': ether_price,
                   'tx_fee': transaction_fee}
        df = df.append(new_row, ignore_index=True)
        print(fr"iteration {iteration} out of {total_iterations}")
        iteration += 1
        if iteration % 1000 == 0:
            df.to_csv(name_of_file + '_' + str(iteration))
        time.sleep(interval_between_iterations)

    return df

In [5]:
fractionalized_nft_gas_df = collect_gas_info(fract_nft_mevs_list, 4)

<ipython-input-4-2938e6d55e2e>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 2130
iteration 1 out of 2130
iteration 2 out of 2130
iteration 3 out of 2130
iteration 4 out of 2130
iteration 5 out of 2130
iteration 6 out of 2130
iteration 7 out of 2130
iteration 8 out of 2130
iteration 9 out of 2130
iteration 10 out of 2130
iteration 11 out of 2130
iteration 12 out of 2130
iteration 13 out of 2130
iteration 14 out of 2130
iteration 15 out of 2130
iteration 16 out of 2130
iteration 17 out of 2130
iteration 18 out of 2130
iteration 19 out of 2130
iteration 20 out of 2130
iteration 21 out of 2130
iteration 22 out of 2130
iteration 23 out of 2130
iteration 24 out of 2130
iteration 25 out of 2130
iteration 26 out of 2130
iteration 27 out of 2130
iteration 28 out of 2130
iteration 29 out of 2130
iteration 30 out of 2130
iteration 31 out of 2130
iteration 32 out of 2130
iteration 33 out of 2130
iteration 34 out of 2130
iteration 35 out of 2130
iteration 36 out of 2130
iteration 37 out of 2130
iteration 38 out of 2130
iteration 39 out of 2130
iteration 

In [ ]:
fractionalized_nft_gas_df = fractionalized_nft_gas_df.to_pickle('fractionalized_nft_gas_df.pkl')